In [ ]:
# default_exp workflows

# workflows
> Execute workflows on Server

In [ ]:
#hide
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# export
from typing import List
import requests

from yx_motor.api import API
from yx_motor.files import Files
from yx_motor.jobs import Jobs


class Workflows:
    "Class for workflow-related API actions"

    def __init__(self, api: API):
        self.api = api
        self.base_endpoint = "workflows/"
        self.files = Files(self.api)
        self.jobs = Jobs(self.api)

    @staticmethod
    def build_schedule_payload(schedule_name: str, asset_id: str):
        return {"schedule": {"name": schedule_name, "assetId": asset_id,}}

    def get_questions(self, asset_id: str):
        """Return question payload for an asset of type analytic app."""
        response = self.api.post(url=f"workflows/{asset_id}/questions")
        return response

    def get_vfs_inputs(self, asset_id: str):
        """Return the list of vfs inputs for a given workflow asset."""
        response = self.api.get(url=f"workflows/{asset_id}/vfsInputs")
        return response

    def update_vfs_inputs(self, asset_id: str, tool_id: str):
        pass

    def get_workflow_dependencies(self, asset_id: str):
        """Return the list of asset dependencies for a given workflow asset"""
        response = self.api.get(url=f"workflows/{asset_id}/dependencies")
        return response

    def update_workflow_dependencies(self, asset_id: str):
        pass

    def add_workflow_dependency(self, asset_id: str):
        pass

    def add_workflow_dependency_connection(self, asset_id: str, tool_id: str):
        pass

    def run_workflow(
        self, asset_id: str, schedule_name: str, vfs_inputs: List[str] = None
    ):
        """Schedule a workflow to be executed instantaneously.
           Returns a schedule object."""
        response = self.api.post(
            url="workflows/run",
            json=self.build_schedule_payload(schedule_name, asset_id),
        )
        return response

    def download_workflow_results(self, schedule_id: str, download_path: str):
        # TODO: Input validation
        workflow_job = self.jobs.get_job(params={"scheduleId": schedule_id})
        # TODO: Need to check the status of the job, for completion
        # and/or error state before trying to get outputs.

        # TODO: Add validation, error handling here
        output_asset_id = self.get_wf_job_output(workflow_job)[0]

        response = self.files.download_file(
            file_uuid=output_asset_id, download_path=download_path
        )

        return response

    @staticmethod
    def get_wf_job_output(wf_job: object):
        jobs_object = wf_job.json()["jobs"]
        job = jobs_object[0]
        outputs = job["outputs"]
        output_asset_id_list = [file["assetId"] for file in outputs]
        return output_asset_id_list

In [ ]:
#hide

######################### Integration Test Scaffolding ############################
from yx_motor.client import Client

from private import server_vars

base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

dev_client = Client(base_url=base_url,
                    login_email=login_email,
                    login_pwd=login_pwd)

workflows = Workflows(dev_client.api)

#TODO Add to a helpers module

from pathlib import Path

def seed_workflow(dev_client: object, 
                  workflow_path: str, 
                  upload_path: str):
    workflow_path = Path(workflow_path)
    workflow_name = workflow_path.name
    
    workflow_response = dev_client.files.upload_file(filename=workflow_path,
                                                     upload_path=f"{upload_path}{workflow_name}",
                                                     conflict_action="MERGE").json()
    
    return workflow_response


questions_workflow_path = Path(r'test_workflows/simple_questions2.yxwz')

workflow_a_path = Path(r'WorkflowA.yxmd')

workflow_b_path = Path(r'WorkflowB.yxmd')

aah_upload_path = r'/Workspaces/Public/'

questions_workflow = seed_workflow(dev_client, 
                                   questions_workflow_path, 
                                   aah_upload_path)

workflow_a_json = seed_workflow(dev_client, 
                                workflow_a_path, 
                                aah_upload_path)

workflow_b_json = seed_workflow(dev_client, 
                                workflow_b_path, 
                                aah_upload_path)

workflow_a_path = workflow_a_json['path']
workflow_a_uuid = workflow_a_json['uuid']
workflow_a_content_id = workflow_a_json['contentId']

workflow_b_path = workflow_b_json['path']
workflow_b_uuid = workflow_b_json['uuid']
workflow_b_content_id = workflow_b_json['contentId']

questions_wf_path = questions_workflow['path']
questions_wf_uuid = questions_workflow['uuid']
questions_wf_content_id = questions_workflow['contentId']

POST sent to: https://hub-beta.demo.alteryx.com/api/v1/authenticate/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Content-Length': '57'}
Response Status: 200
POST sent to: https://hub-beta.demo.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/simple_questions2.yxwz', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A0a2fb66e-de46-47db-9be0-9245802f96e7.wGhOf6H63PZh2aEFEgk9xHicI9irLpJBYGONAvdBTSU', 'Content-Length': '22646'}
Response Status: 200
POST sent to: https://hub-beta.demo.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', '

# Why is get questions broken? Look at API docs

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.get_questions)

<h4 id="Workflows.get_questions" class="doc_header"><code>Workflows.get_questions</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.get_questions</code>(**`asset_id`**:`str`)

Return question payload for an asset of type analytic app.

In [ ]:
workflows = Workflows(dev_client.api)

In [ ]:
asset_uid = questions_wf_uuid

In [ ]:
workflows.get_questions(asset_id=asset_uid).json()

POST sent to: https://ec2-44-234-86-100.us-west-2.compute.amazonaws.com/api/v1/workflows/061ccef2-42bc-45f6-b6ed-77cc6a211dc5/questions
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Ae5792dbf-36ad-4fb4-bffd-2d3d449b08c4.OTu4BC6lMigmko%2FjBZ9k%2BHb8h9aLG7DZjlNS22n0uNk', 'Content-Length': '0'}
Response Status: 500


{'requestUuid': '614951e1-6dcb-4026-a296-f381b73e42bf',
 'errors': [{'code': '',
   'message': 'WorkflowOrchestrator.getQuestions() with args: [ { "requestUuid": "614951e1-6dcb-4026-a296-f381b73e42bf","auditEvents": "[  ]","privateSession": "{ NESTED OBJECT }","privateSiteId": "f6a2add6-7f4b-4c62-9b21-b9959faac597","clientIp": "::ffff:66.35.44.241" },"061ccef2-42bc-45f6-b6ed-77cc6a211dc5",undefined,undefined,undefined]'},
  {'code': '',
   'message': 'EngineWorkerService.getQuestions() with args: [ { "requestUuid": "614951e1-6dcb-4026-a296-f381b73e42bf","auditEvents": "[  ]","privateSession": "{ NESTED OBJECT }","privateSiteId": "f6a2add6-7f4b-4c62-9b21-b9959faac597","clientIp": "::ffff:66.35.44.241" }, { "created": "\\"2020-07-01T21:47:31.549Z\\"","versionCreated": "\\"2020-07-01T21:47:31.549Z\\"","folderType":  [[[ this argument failed to process ]]],"links":  [[[ this argument failed to process ]]],"metadata": "{ NESTED OBJECT }","metaHash": "f1425e61763e6a7fae79 ... 8722de8a69d0bc6

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.get_vfs_inputs)

<h4 id="Workflows.get_vfs_inputs" class="doc_header"><code>Workflows.get_vfs_inputs</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.get_vfs_inputs</code>(**`asset_id`**:`str`)

Return the list of vfs inputs for a given workflow asset.

In [ ]:
workflows = Workflows(dev_client.api)
workflows.get_vfs_inputs(asset_id=asset_uid).json()

GET sent to: https://ec2-44-234-86-100.us-west-2.compute.amazonaws.com/api/v1/workflows/ed2085aa-6c6f-4e45-b5ce-5ccc2bfa6771/vfsInputs
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A174b59ad-dd9c-47e4-ace7-e6abf63094e7.4UsHXVoFG%2B5AWeDcp0lfsE4BF59GfxlaPo3tDz1G5qM'}
Response Status: 200


[]

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.get_workflow_dependencies)

<h4 id="Workflows.get_workflow_dependencies" class="doc_header"><code>Workflows.get_workflow_dependencies</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.get_workflow_dependencies</code>(**`asset_id`**:`str`)

Return the list of asset dependencies for a given workflow asset

In [ ]:
workflows = Workflows(dev_client.api)
workflows.get_workflow_dependencies(asset_id=asset_uid).json()

GET sent to: https://ec2-44-234-86-100.us-west-2.compute.amazonaws.com/api/v1/workflows/ed2085aa-6c6f-4e45-b5ce-5ccc2bfa6771/dependencies
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A174b59ad-dd9c-47e4-ace7-e6abf63094e7.4UsHXVoFG%2B5AWeDcp0lfsE4BF59GfxlaPo3tDz1G5qM'}
Response Status: 200


[]

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.run_workflow)

<h4 id="Workflows.run_workflow" class="doc_header"><code>Workflows.run_workflow</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.run_workflow</code>(**`asset_id`**:`str`, **`schedule_name`**:`str`, **`vfs_inputs`**:`List`\[`str`\]=*`None`*)

Schedule a workflow to be executed instantaneously.
Returns a schedule object.

**Arguments**:

- asset_id: Unique VFS identifier for the workflow you want to execute.
- schedule_name: Optional: Name of the schedule that will be created when workflow execution is triggered.
- vfs_inputs: TODO, need to research this more and test it.

In [ ]:
asset_uid = questions_wf_uuid

In [ ]:
workflows = Workflows(dev_client.api)
workflows.run_workflow(asset_id=asset_uid,
                       schedule_name="jp_client_test").json()

POST sent to: https://ec2-44-234-86-100.us-west-2.compute.amazonaws.com/api/v1/workflows/run
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Ae5792dbf-36ad-4fb4-bffd-2d3d449b08c4.OTu4BC6lMigmko%2FjBZ9k%2BHb8h9aLG7DZjlNS22n0uNk', 'Content-Length': '91'}
Response Status: 200


{'scheduleId': 'c4e886d1-423b-4cb4-a985-195ac511d108',
 'name': 'jp_client_test',
 'userId': '88eec279-7769-414e-94ae-d0fd9f83a686',
 'assetId': '061ccef2-42bc-45f6-b6ed-77cc6a211dc5',
 'assetVersion': None,
 'runAsId': None,
 'toolConnections': None,
 'status': 'active',
 'startDate': None,
 'endDate': None,
 'lastRun': None,
 'nextRun': None,
 'frequencyInterval': 'manual',
 'frequencyRule': None,
 'type': 'immediate',
 'isDisabled': False,
 'isDeleted': False,
 'comment': None,
 'siteId': 'f6a2add6-7f4b-4c62-9b21-b9959faac597',
 'parameters': {'runMode': 0},
 'analyticAppAnswers': None,
 'inputFiles': None,
 'creationDate': '2020-07-01T21:49:02.748Z',
 'lastUpdate': '2020-07-01T21:49:02.748Z',
 'outputLocation': None,
 'timezone': 'UTC',
 'utcOffset': 0,
 'priority': 50}

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.download_workflow_results)

<h4 id="Workflows.download_workflow_results" class="doc_header"><code>Workflows.download_workflow_results</code><a href="__main__.py#L61" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.download_workflow_results</code>(**`schedule_id`**:`str`, **`download_path`**:`str`)



In [ ]:
workflows = Workflows(dev_client.api)
response = workflows.run_workflow(asset_id=asset_uid,
                       schedule_name="jp_client_test").json()

POST sent to: https://ec2-44-234-86-100.us-west-2.compute.amazonaws.com/api/v1/workflows/run
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A174b59ad-dd9c-47e4-ace7-e6abf63094e7.4UsHXVoFG%2B5AWeDcp0lfsE4BF59GfxlaPo3tDz1G5qM', 'Content-Length': '91'}
Response Status: 200


In [ ]:
schedule_id = response['scheduleId']

In [ ]:
workflows.download_workflow_results(schedule_id=schedule_id, 
                                    download_path='wf_out_test.csv')

GET sent to: https://ec2-44-234-86-100.us-west-2.compute.amazonaws.com/api/v1/jobs/?scheduleId=22317f84-5a10-4633-8079-64c77f1dc740
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A174b59ad-dd9c-47e4-ace7-e6abf63094e7.4UsHXVoFG%2B5AWeDcp0lfsE4BF59GfxlaPo3tDz1G5qM'}
Response Status: 200
GET sent to: https://ec2-44-234-86-100.us-west-2.compute.amazonaws.com/api/v1/files/content?id=ddc6c43d-f7ee-4d76-9c9b-320afe74b292
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A174b59ad-dd9c-47e4-ace7-e6abf63094e7.4UsHXVoFG%2B5AWeDcp0lfsE4BF59GfxlaPo3tDz1G5qM'}
Response Status: 200


<Response [200]>

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 01_client.ipynb.
Converted 02_jobs.ipynb.
Converted 03_api.ipynb.
Converted 04_authenticate.ipynb.
Converted 05_files.ipynb.
Converted 06_workflows.ipynb.
Converted index.ipynb.
Converted integration_tests.ipynb.


In [ ]:
from nbdev.sync import script2notebook
script2notebook()